# Reading  the Json file.

In [2]:
import json

with open("/content/json-fixer.json", "r", encoding='utf-8') as f_file:
    data = f_file.read()
    print(data)

{
    "session_name": "test",
    "session_description": "test",
    "design_state_data": {

      "session_info" : {
        "project_id": "1",
        "experiment_id": "kkkk-11",
        "dataset":"iris_modified.csv",
        "session_name": "test",
        "session_description": "test"
        },

      "target": {
        "prediction_type": "Regression",
        "target": "petal_width",
        "type":"regression",
        "partitioning": true
      },
      "train": {
        "policy": "Split the dataset",
        "time_variable": "sepal_length",
        "sampling_method": "No sampling(whole data)",
        "split": "Randomly",
        "k_fold": false,
        "train_ratio": 0,
        "random_seed": 0
      },
      "metrics": {
        "optomize_model_hyperparameters_for": "AUC",
        "optimize_threshold_for": "F1 Score",
        "compute_lift_at": 0,
        "cost_matrix_gain_for_true_prediction_true_result": 1,
        "cost_matrix_gain_for_true_prediction_false_result": 0,

In [74]:
df = pd.read_csv('/content/iris.csv')
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


# Generic code

In [27]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mean_absolute_error
from sklearn.metrics import mean_squared_error as mean_squared_error
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, StratifiedKFold, KFold

# Load the JSON configuration
with open("/content/json-fixer.json") as f:
    config = json.load(f)

def extract_session_info(config):
    session_info = config['design_state_data']['session_info']
    project_id = session_info['project_id']
    experiment_id = session_info['experiment_id']
    dataset = session_info['dataset']
    session_name = session_info['session_name']
    session_description = session_info['session_description']

    return project_id, experiment_id, dataset, session_name, session_description


# data is the dataset containing the features and target variable
data = pd.read_csv('/content/iris.csv')
# You can replace 'data' with your actual dataset
data.drop('species', axis=1, inplace=True)
# Target and feature handling
prediction_type = config["design_state_data" ]['target']['prediction_type'],
target_column = config["design_state_data" ]['target']['target']
X = data.drop(target_column, axis=1)
y = data[target_column]
type_ = config["design_state_data" ]['target']['type']
partitioning = config["design_state_data" ]['target']['partitioning']

# Assuming 'X_train', 'X_test', 'y_train', 'y_test' are the training and test features and target, respectively
# Splitting the data for training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# Handling features

# Function to handle a specific feature based on the configuration
def handle_feature(config, data):
    # Extract feature name and details from the configuration
    feature_name = config['feature_name']
    is_selected = config['is_selected']
    feature_variable_type = config['feature_variable_type']

    if not is_selected:
        # If the feature is not selected, simply drop it from the data
        data.drop(feature_name, axis=1, inplace=True)
        return data

    if feature_variable_type == 'numerical':
        # Numerical feature handling
        numerical_handling = config['feature_details']['numerical_handling']

        if numerical_handling == 'Keep as regular numerical feature':
            # No special handling required
            pass
        elif numerical_handling == 'Rescale':
            rescaling_method = config['feature_details']['rescaling']
            if rescaling_method == 'Min-Max scaling':
                scaler = MinMaxScaler()
                # Apply Min-Max scaling to the feature
                data[feature_name] = scaler.fit_transform(data[feature_name].values.reshape(-1, 1))
            elif rescaling_method == 'Standardization':
                scaler = StandardScaler()
                # Apply Standardization to the feature
                data[feature_name] = scaler.fit_transform(data[feature_name].values.reshape(-1, 1))
            else:
                # Implement other rescaling methods if needed
                pass

        # Handle missing values
        missing_values = config['feature_details']['missing_values']
        if missing_values == 'Impute':
            impute_with = config['feature_details']['impute_with']
            impute_value = config['feature_details']['impute_value']
            if impute_with == 'Average of values':
                # Impute missing values with the average of non-missing values
                data[feature_name].fillna(data[feature_name].mean(), inplace=True)
            elif impute_with == 'custom':
                # Impute missing values with a custom value
                data[feature_name].fillna(impute_value, inplace=True)

    return data



# Feature Reduction


def perform_feature_reduction(config, X_train, y_train, X_test):
    feature_reduction_method = config['feature_reduction_method']

    if feature_reduction_method == 'No Reduction':
        # Keep a fixed number of features
        num_of_features_to_keep = config['No Reduction']['num_of_features_to_keep']
        selected_features = X_train.columns[:num_of_features_to_keep]

    elif feature_reduction_method == 'Correlation with target':
        # Perform feature selection based on correlation with target
        is_selected = config['Correlation with target']['is_selected']
        if is_selected:
            num_of_features_to_keep = config['Correlation with target']['num_of_features_to_keep']
            selector = SelectKBest(score_func=f_regression, k=num_of_features_to_keep)
            selector.fit(X_train, y_train)
            selected_features = X_train.columns[selector.get_support()]

    elif feature_reduction_method == 'Tree-based':
        # Perform feature selection based on feature importances from a tree-based model
        is_selected = config['Tree-based']['is_selected']
        if is_selected:
            num_of_features_to_keep = config['Tree-based']['num_of_features_to_keep']
            depth_of_trees = config['Tree-based']['depth_of_trees']
            num_of_trees = config['Tree-based']['num_of_trees']

            '''model = RandomForestRegressor(n_estimators=num_of_trees, max_depth=depth_of_trees, random_state=0)
            model.fit(X_train, y_train)

            importances = model.feature_importances_
            feature_importance_df = pd.DataFrame({'feature': X_train.columns, 'importance': importances})
            feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
            selected_features = feature_importance_df.head(num_of_features_to_keep)['feature'].tolist()'''

    elif feature_reduction_method == 'Principal Component Analysis':
        # Perform dimensionality reduction using PCA
        is_selected = config['Principal Component Analysis']['is_selected']
        if is_selected:
            num_of_features_to_keep = config['Principal Component Analysis']['num_of_features_to_keep']
            pca = PCA(n_components=num_of_features_to_keep)
            X_train_reduced = pca.fit_transform(X_train)
            X_test_reduced = pca.transform(X_test)
            selected_features = X_train.columns[:num_of_features_to_keep]
            X_train = pd.DataFrame(X_train_reduced, columns=selected_features)
            X_test = pd.DataFrame(X_test_reduced, columns=selected_features)

    else:
        # Implement other feature reduction methods if needed
        selected_features = X_train.columns

    return X_train, X_test, selected_features


# Model training and evaluation
def train_and_evaluate_model(config, X_train, y_train, X_test, y_test):
    model_name = config['model_name']
    is_selected = config['is_selected']

    if not is_selected:
        print(f"Skipping {model_name} as it is not selected.")
        return

    if model_name == 'Random Forest Classifier':
        model = RandomForestClassifier(n_estimators=config['max_trees'],
                                       min_samples_leaf=config['min_samples_per_leaf_min_value'],
                                       max_samples_leaf=config['min_samples_per_leaf_max_value'],
                                       max_depth=config['max_depth'],
                                       random_state=config['parallelism'])
    elif model_name == 'LinearRegression':
        model = LinearRegression(n_jobs=config['parallelism'],
                                 max_iter=config['max_iter'],
                                 alpha=np.linspace(config['min_regparam'], config['max_regparam']),
                                 l1_ratio=np.linspace(config['min_elasticnet'], config['max_elasticnet']))
    elif model_name == 'LogisticRegression':
        model = LogisticRegression()
    elif model_name == 'Random Forest Regressor':
        model = RandomForestRegressor(n_estimators=config['max_trees'],
                                      min_samples_leaf=config['min_samples_per_leaf_min_value'],
                                      max_samples_leaf=config['min_samples_per_leaf_max_value'],
                                      max_depth=config['max_depth'],
                                      random_state=config['parallelism'])
    else:
        print(f"Unknown model name: {model_name}")
        return

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

#hyperparameters

def optimize_hyperparameters(config, model, X_train, y_train):
    # Extract hyperparameter configuration details
    search_method = config['search_method']
    grid_search_config = config['Grid Search']

    # Hyperparameter tuning with Grid Search
    if search_method == 'Grid Search':
        # Extract grid search configuration details
        is_selected = grid_search_config['is_selected']
        shuffle_grid = grid_search_config['shuffle_grid']
        random_state = grid_search_config['random_state']
        max_iterations = grid_search_config['max_iterations']
        max_search_time = grid_search_config['max_search_time']
        cross_validation_strategy = grid_search_config['cross_validation_strategy']

        # Create a dictionary of hyperparameters for the grid search
        hyperparameter_grid = { 'n_estimators': [50, 100, 150],
                               'max_depth': [None, 10, 20],
                               'min_samples_split': [2, 5],
                              'min_samples_leaf': [1, 2]}

        # Create a cross-validation strategy based on the specified configuration
        if cross_validation_strategy == 'Time-based K-fold (with overlap)':
            time_kfold_config = grid_search_config['Time-based K-fold (with overlap)']
            is_selected_kfold = time_kfold_config['is_selected']
            num_of_folds = time_kfold_config['num_of_folds']
            split_ratio = time_kfold_config['split_ratio']
            stratified = time_kfold_config['stratified']

            if is_selected_kfold:
                if stratified:
                    cv = StratifiedKFold(n_splits=num_of_folds, shuffle=shuffle_grid, random_state=random_state)
                else:
                    cv = KFold(n_splits=num_of_folds, shuffle=shuffle_grid, random_state=random_state)
            else:
                cv = None
        else:
            # Implement other cross-validation strategies if needed
            cv = None

        # Initialize the GridSearchCV with the model and hyperparameter grid
        grid_search = GridSearchCV(model, hyperparameter_grid, cv=cv, n_jobs=-1, verbose=2)

        # Perform grid search to find the best hyperparameters
        grid_search.fit(X_train, y_train)

        # Get the best model with optimized hyperparameters
        best_model = grid_search.best_estimator_

        return best_model


In [17]:
project_id, experiment_id, dataset, session_name, session_description = extract_session_info(config)
print("Project ID:", project_id)
print("Experiment ID:", experiment_id)
print("Dataset:", dataset)
print("Session Name:", session_name)
print("Session Description:", session_description)

Project ID: 1
Experiment ID: kkkk-11
Dataset: iris_modified.csv
Session Name: test
Session Description: test


In [57]:
    if prediction_type == 'Regression':
        # Evaluation metrics for regression

        # Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_test, y_pred)

        # Mean Squared Error (MSE)
        mse = mean_squared_error(y_test, y_pred)



    elif prediction_type == 'Classification':
        # Evaluation metrics for classification

        # Accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # F1-score
        f1 = f1_score(y_test, y_pred)
        print(f"{model} - F1-score:", f1)

# Testing on given data set.

In [66]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

print(f"{model} - Mean Absolute Error (MAE):", mae)
print(f"{model} - Mean Squared Error (MSE):", mse)

LinearRegression() - Mean Absolute Error (MAE): 0.1693268164410859
LinearRegression() - Mean Squared Error (MSE): 0.051607270608851764
